<img src="ubiops_logo.svg" width="80">

# Quickstart
Deploy a simple float-to-float deployment in UbiOps using the python client library.


The first step is to [download](https://storage.googleapis.com/ubiops/example-deployment-packages/example_deployment_package.zip) a prepared deployment package (the deployment code).

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/example_deployment_package.zip -o example_deployment_package.zip

Add your API token and project name. You can also adapt the deployment name and deployment version name or leave the default values. Afterwards we initialize the client library, which establishes the connection with UbiOps.

In [ ]:
API_TOKEN = 'Token <YOUR_API_KEY>'
PROJECT_NAME = '<YOUR_PROJECT_NAME>'
DEPLOYMENT_NAME = 'simple-tutorial'
DEPLOYMENT_VERSION = 'v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration(host="https://api.ubiops.com/v2.1")
configuration.api_key['Authorization'] = API_TOKEN

Here we open the connection with the UbiOps API Client.

In [ ]:
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy

### Create a deployment

In [ ]:
deployment_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='A simple deployment that multiplies the input float by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[{'name': 'input', 'data_type': 'double'}],
    output_fields=[{'name': 'output', 'data_type': 'double'}],
    labels={"demo": "quickstart"}
)

deployment = api.deployments_create(project_name=PROJECT_NAME, data=deployment_template)
print(deployment)

### Create a deployment version

<div class="alert alert-block alert-info">
<b>Request Retention Mode:</b>
Use <code>request_retention_mode='metadata'</code> or <code>request_retention_mode='full'</code> to store all requests. <u>Only then you will be able to retrieve them later</u>.
</div>

In [ ]:
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.7',
    instance_type='256mb',
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800, # = 30 minutes
    request_retention_mode='full'
)

version = api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)
print(version)

Upload the prepared deployment file to the created deployment version. This will create a revision and starts a build.

In [ ]:
upload_response = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='example_deployment_package.zip'
)
print(upload_response)

Check if deployment is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

The version is now available.

In [ ]:
api.deployment_versions_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
).status

## Requests

<div class="alert alert-block alert-info">

<b>Direct requests:</b> Direct (<i>synchronous</i>) requests have a maximum duration of <i>1 hour</i>. The connection stays open until the request is finished. The results are in the response of the request.

<b>Batch requests:</b> Batch (<i>asynchronous</i>) requests have a maximum duration of <i>48 hours</i>. A request is created with status <i>pending</i> and the request id is returned. With the request id you can obtain the results later. <br/>Please, note that it is only possible to retrieve the results later if you used <code>request_retention_mode='full'</code> upon deployment version creation.<br/>

### Creating a direct request
Make sure the deployment is in 'available' state before performing deployment requests.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

In [ ]:
data = {'input': 123}
request_1 = api.deployment_version_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    data=data
)
print(request_1)

Result:

In [ ]:
print(request_1.result)

It's possible to make a request to the *default* deployment version. The first created version automatically becomes the *default* version, and can be set to another version later. 

In [ ]:
data = {'input': 123}
request_2 = api.deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=data
)
print(request_2)

Result:

In [ ]:
print(request_2.result)

### Creating batch requests

Create the request(s) and retrieve the request id(s)

In [ ]:
data = {'input': 123}
request_batch = api.batch_deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=[data]
)
print(request_batch)

Retrieve the results using the request id

<div class="alert alert-block alert-info">
Please, note that it is only possible to retrieve the results later if you used <code>request_retention_mode='full'</code> upon deployment version creation.
</div>

In [ ]:
request_result = api.deployment_requests_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    request_id=request_batch[0].id
)
print(request_result)

***

# Pipelines

Multiple deployments can be chained using a *pipeline*. In this tutorial, we will chain the deployment created above with a new deployment. Just for demo purposes, we will use the same deployment package (the same code) for both deployments in the pipeline. Therefore, the pipeline will contain two modelling steps: each step will multiply the input by a random number.

## Create another deployment and version
Normally you would like to chain different deployments. However, just for demo purposes, we will use the same deployment package as used above.

Create the deployment and version

In [ ]:
DEPLOYMENT_NAME2 = f"{DEPLOYMENT_NAME}-2"
deployment_template2 = deployment_template
deployment_template2.name = DEPLOYMENT_NAME2

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template2
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    data=version_template
)

Upload the zip

In [ ]:
upload_response2 = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    version=DEPLOYMENT_VERSION,
    file='example_deployment_package.zip'
)
print(upload_response2)

Wait for the deployment to be ready

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':    
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME2,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response2.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

## Create a pipeline and version

In [ ]:
PIPELINE_NAME = f"{DEPLOYMENT_NAME}-pipeline"
PIPELINE_VERSION = DEPLOYMENT_VERSION

In [ ]:
pipeline_template = ubiops.PipelineCreate(
    name=PIPELINE_NAME,
    description='A simple pipeline that multiplies the input float two times by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[{'name': 'input', 'data_type': 'double'}],
    output_fields=[{'name': 'output', 'data_type': 'double'}],
    labels={"demo": "quickstart"}
)

api.pipelines_create(project_name=PROJECT_NAME, data=pipeline_template)

<div class="alert alert-block alert-info">
    
<b>Request Retention Mode:</b>
Use <code>request_retention_mode='metadata'</code> (request info) or <code>request_retention_mode='full'</code> (request info + input/output data) to store all requests to the pipeline version. Only then you will be able to retrieve them later.
</div>

In [ ]:
pipeline_template = ubiops.PipelineVersionCreate(
    version=PIPELINE_VERSION,
    request_retention_mode='full',
    objects=[
        # object 1
        # Refer a specific version of deployment1
        {
            'name': DEPLOYMENT_NAME,
            'reference_name': DEPLOYMENT_NAME,
            'version': DEPLOYMENT_VERSION
        },
        # object 2
        # Refer default version of deployment2
        {
            'name': DEPLOYMENT_NAME2,
            'reference_name': DEPLOYMENT_NAME2
        }
    ],
    attachments=[
        # start -> deployment1
        {
            'destination_name': DEPLOYMENT_NAME,
            'sources': [{
                'source_name': 'pipeline_start',
                'mapping': [{
                    "source_field_name": 'input',
                    'destination_field_name': 'input'
                }]
            }]
        },
        # deployment1 -> deployment2
        {
            'destination_name': DEPLOYMENT_NAME2,
            'sources': [{
                'source_name': DEPLOYMENT_NAME,
                'mapping': [{
                    "source_field_name": 'output',
                    'destination_field_name': 'input'
                }]
            }]
        },
        # deployment2 -> end
        {
            'destination_name': 'pipeline_end',
            'sources': [{
                'source_name': DEPLOYMENT_NAME2,
                'mapping': [{
                    "source_field_name": 'output',
                    'destination_field_name': 'output'
                }]
            }]
        }
    ]
)

api.pipeline_versions_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=pipeline_template)

## Requests

### Creating a direct request

In [ ]:
data = {'input': 123}
pipeline_result = api.pipeline_version_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    version=PIPELINE_VERSION,
    data=data
)
print(pipeline_result)

It's also possible to make a request to the *default* pipeline version. Like for deployments, the first created pipeline version automatically becomes the *default* version, and can be set to another version later.

In [ ]:
data = {'input': 123}
pipeline_result = api.pipeline_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=data
)
print(pipeline_result)

### Creating batch requests

In [ ]:
data = {'input': 123}
pipeline_batch = api.batch_pipeline_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=[data]
)
print(pipeline_batch)

<div class="alert alert-block alert-info">
Please, note that it is only possible to retrieve the results later if you use <code>request_retention_mode='full'</code>.
</div>

In [ ]:
pipeline_result = api.pipeline_requests_get(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    request_id=pipeline_batch[0].id
)
print(pipeline_result)

## Cleanup
Delete created deployment.

In [ ]:
api.pipelines_delete(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME
)

In [ ]:
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME
)
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2
)

## Close connection
Close the connection with the UbiOps API client.

In [ ]:
client.close()